In [ ]:
source("diabetes_analysis_v05.R")

In [ ]:
honardoost  <- readRDS("../data/published_data/Honardoost_2024/T1D_Seurat_Object_Final.rds")

In [ ]:
honardoost

In [ ]:
honardoost@assays$Projection

In [ ]:
DimPlot(honardoost)

In [ ]:
honardoost <- NormalizeData(honardoost, verbose = FALSE)
honardoost <- FindVariableFeatures(honardoost, nfeatures = 1000, verbose = FALSE)
honardoost <- ScaleData(honardoost, verbose = FALSE)
honardoost <- RunPCA(honardoost, npcs = 20, verbose = FALSE)
honardoost <- RunUMAP(honardoost, reduction = "pca", dims = 1:20)

In [ ]:
DimPlot(honardoost, raster = T, label  = T)

In [ ]:
honardoost <- FindNeighbors(honardoost, dims = 1:12)
honardoost <- FindClusters(honardoost, resolution = 0.3)

In [ ]:
DimPlot(honardoost, raster = T, label = T)

In [ ]:
honardoostT  <- subset(honardoost, seurat_clusters %in% c(0,1,3,5,6))

In [ ]:
honardoostT <- NormalizeData(honardoostT, verbose = FALSE)
honardoostT <- FindVariableFeatures(honardoostT, nfeatures = 1000, verbose = FALSE)
honardoostT <- ScaleData(honardoostT, verbose = FALSE)
honardoostT <- RunPCA(honardoostT, npcs = 20, verbose = FALSE)
honardoostT <- RunUMAP(honardoostT, reduction = "pca", dims = 1:20)
honardoostT <- FindNeighbors(honardoostT, dims = 1:12)
honardoostT <- FindClusters(honardoostT, resolution = 0.3)

In [ ]:
saveRDS(honardoostT, "../data/published_data/Honardoost_2024/honardoost_tcells.rds")

In [ ]:
honardoostT  <- readRDS("../data/published_data/Honardoost_2024/honardoost_tcells.rds")

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(honardoostT, raster = T, label = T)

In [ ]:
rm(honardoost)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
FeaturePlot(honardoostT, features = "CD8A", min.cutoff = 0, raster = T)

In [ ]:
FeaturePlot(honardoostT, features = "CD4", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "FOXP3", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "SELL", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "CCL5", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "GNLY", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "CD3D", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "TRAC", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "CD19", min.cutoff = 0, raster = T)
FeaturePlot(honardoostT, features = "MS4A1", min.cutoff = 0, raster = T)

In [ ]:
FeaturePlot(honardoostT, features = "CD8A", min.cutoff = 0, raster = T)

In [ ]:
FeaturePlot(honardoostT, features = "TRGC1", min.cutoff = 0, raster = T)


In [ ]:
honardoostT@meta.data  %>% colnames

In [ ]:
DimPlot(honardoostT, raster = T, label = T, group.by = "Cluster_Annotation_Merged")

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 16)

DimPlot(honardoostT, raster = T, label = T, group.by = "Sample_ID")

In [ ]:
honardoostT <- FindClusters(honardoostT, resolution = 0.23)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
DimPlot(honardoostT, raster = T, label = T)

In [ ]:
mrk  <- FindAllMarkers(honardoostT, logfc.threshold = log(2))

In [ ]:
mrk  %>% filter(cluster ==0 & avg_log2FC >0)

In [ ]:
FeaturePlot(honardoostT, features = "RP11-36C20.1", min.cutoff = 0, raster = T)


In [ ]:
options(repr.plot.width = 20)
VlnPlot(honardoostT, features = c("percent.mt", "percent.rp", "nCount_RNA", "nFeature_RNA"), ncol = 4, raster = F)

In [ ]:
options(repr.plot.width = 6)

FeaturePlot(honardoostT, features = "FOXP3", min.cutoff = 0, raster = T, max.cutoff = 1)


In [ ]:
    honardoostT$Sample_ID  %>% table

In [ ]:
honardoostT@meta.data  %>% colnames

In [ ]:
honardoostT$COND  %>% table

In [ ]:
md  <- honardoostT@meta.data  %>% dplyr::select(Sample_ID, HLA_Haplotypes, COND)

In [ ]:
avgexp = AverageExpression(honardoostT, features = c("NOTCH4","TABP","TAP2","LMP7","TAP1","LMP2","LST1",
                                                     "LTB","TNF","LTA","NFKBIL1", "BTN3A2","BTN3A1"),
                           return.seurat = F, group.by = "Sample_ID", 
                          assay = "RNA")

btn_etc  <- t(avgexp$RNA)  %>% as.data.frame()  %>% 
rownames_to_column("Sample_ID")  %>% 
pivot_longer(!Sample_ID, names_to = "gene", values_to = "expression")  %>% 
left_join(md)  %>% unique

btn_etc

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)

btn_etc  %>% filter(gene == "BTN3A2")  %>% 
ggplot(aes(x = HLA_Haplotypes,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = COND)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
            axis.text = element_text(angle = 90),
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA CD4 T cells")

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7)

btn_etc  %>% filter(gene == "BTN3A2")  %>% 
ggplot(aes(x = HLA_Haplotypes,
             y = expression)) +
   geom_dotplot(binaxis='y', stackdir='center', dotsize = 0) + 
   geom_boxplot(outlier.shape = NA) +
    geom_jitter(binaxis='y', position=position_jitter(width = 0.1, height = 0.01), 
                size = 2, stackdir='center', aes(color = COND)) + 
    facet_wrap(~gene, scales = "free") +
  theme_classic() + xlab("") +  
 xlab("") + ylab("Value") +
   scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
  ggpubr::stat_compare_means(label.x = 1.2, label.y.npc = "top", size = 5, vjust = 0.3, label = "p.format") + 
      theme(plot.title = element_text(hjust = 0.5, size = 18),
          axis.line = element_line(colour = "black"), 
            axis.text = element_text(angle = 90),
        axis.ticks = element_line(colour = "black")) +
  ylim(0,NA) + ggtitle("Avg Expression by HLA honardoost")

In [ ]:
DimPlot(honardoostT, raster = T, label = T, group.by = "Cluster_Annotation_Merged")
DimPlot(honardoostT, raster = T, label = T, cells.highlight ´group.by = "Cluster_Annotation_Merged")

In [ ]:
treg  <- subset(honardoostT, Cluster_Annotation_Merged == "T_reg")

In [ ]:
DimPlot(treg)

In [ ]:
FeaturePlot(treg, features = "FOXP3")

In [ ]:
treg <- NormalizeData(treg, verbose = FALSE)
treg <- FindVariableFeatures(treg, nfeatures = 1000, verbose = FALSE)
treg <- ScaleData(treg, verbose = FALSE)
treg <- RunPCA(treg, npcs = 20, verbose = FALSE)
treg <- RunUMAP(treg, reduction = "pca", dims = 1:20)
treg <- FindNeighbors(treg, dims = 1:12)
treg <- FindClusters(treg, resolution = 0.3)

In [ ]:
DimPlot(treg)

FeaturePlot(treg, features = "FOXP3")

In [ ]:
FeaturePlot(treg, features = "FOXP3")
FeaturePlot(treg, features = "CD226")
FeaturePlot(treg, features = "CCR7")
FeaturePlot(treg, features = "TNFRSF9")
FeaturePlot(treg, features = "CCR10")
FeaturePlot(treg, features = "MKI67")

In [ ]:
FeaturePlot(treg, features = "CTLA4")
FeaturePlot(treg, features = "TIGIT")

In [ ]:
FeaturePlot(treg, features = "GZMK")

In [ ]:
mrk_treg  <- FindAllMarkers(treg)

In [ ]:
DimPlot(treg, label = T)


In [ ]:
FeaturePlot(treg, features = "IL32")
FeaturePlot(treg, features = "RP11-864N7.2")


In [ ]:
FeaturePlot(treg, features = "TRAC")


In [ ]:
mrk_treg  %>% filter(cluster == 0 & avg_log2FC >0)

In [ ]:
mrk_treg  %>% filter(cluster == 3 & avg_log2FC >0)

In [ ]:
DimPlot(treg, label = T, group.by = "COND")


In [ ]:
treg1  <- subset(treg, seurat_clusters == 1)

# Proper dataset evaluation

In [ ]:
honardoostT  <- readRDS("../data/published_data/Honardoost_2024/honardoost_tcells.rds")

In [ ]:
DimPlot(honardoostT)

In [ ]:
mtx  <- honardoostT@assays$RNA@counts[which(!(grepl(rownames(honardoostT@assays$RNA@counts), pattern = "MT-")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\.")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^MIR")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^MT[RN]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "-AS")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "LINC")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "ORF")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^TR[AB]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "^RP[LS]")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\-")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "\\.")|
             grepl(rownames(honardoostT@assays$RNA@counts), pattern = "mir")) ),]

In [ ]:
mtx

In [ ]:
honar2  <- CreateSeuratObject(counts = mtx, meta.data = honardoostT@meta.data, min.cells = 3, min.features = 200)

In [ ]:
honar2 <- NormalizeData(honar2, verbose = FALSE)
honar2 <- FindVariableFeatures(honar2, nfeatures = 1000, verbose = FALSE)
honar2 <- ScaleData(honar2, verbose = FALSE)
honar2 <- RunPCA(honar2, npcs = 20, verbose = FALSE)
honar2 <- RunUMAP(honar2, reduction = "pca", dims = 1:20)
honar2 <- FindNeighbors(honar2, dims = 1:12)
honar2 <- FindClusters(honar2, resolution = 0.3)

In [ ]:
DimPlot(honar2)

In [ ]:
FeaturePlot(honar2, features = "CD4")
FeaturePlot(honar2, features = "CD8A")
FeaturePlot(honar2, features = "CD3D")

In [ ]:
FeaturePlot(honar2, features = "FOXP3")
FeaturePlot(honar2, features = "CD226")
FeaturePlot(honar2, features = "CCR7")
FeaturePlot(honar2, features = "TNFRSF9")
FeaturePlot(honar2, features = "CCR10")
FeaturePlot(honar2, features = "MKI67")

In [ ]:
DimPlot(honar2, group.by = "Cluster_Annotation_Merged")

In [ ]:
DimPlot(honar2)

In [ ]:
mrk  <- FindAllMarkers(honar2)

In [ ]:
mrk

In [ ]:
options(repr.plot.height = 15, repr.plot.width = 20)
for(i in 0:6){
    print(paste("Cluster",i))
    genes  <- mrk  %>% dplyr::filter(cluster == i)  %>% slice_head(n = 12)  %>% pull(gene)
    print(FeaturePlot(honar2, features = genes, ncol = 4))
    }

In [ ]:
print(FeaturePlot(honar2, features = c("TRGC1","TRDC","TRGV9","TRDC1","GZMB","GZMK"), ncol = 4))

In [ ]:
gd  <- subset(honar2, seurat_clusters == 4)

In [ ]:
colnames(gd@meta.data)

In [ ]:
gd@meta.data  <- gd@meta.data  %>% separate(Sample_ID, into = c("Disease", "Patient_ID"), sep = "-", remove = F)

In [ ]:
honar2@meta.data  <- honar2@meta.data  %>% separate(Sample_ID, into = c("Disease", "Patient_ID"), sep = "-", remove = F)

In [ ]:
DimPlot(honar2, group.by = "LIB")

In [ ]:
DimPlot(gd, group.by = "Disease")

In [ ]:
FeaturePlot(gd, features = "GZMB")

In [ ]:
FeaturePlot(gd, features = "GZMK")

# STACAS

In [ ]:
honar2  <- SplitObject(honar2, split.by = "LIB")

In [ ]:
# normalize and identify variable features for each dataset independently
merged.list <- lapply(X = honar2, FUN = function(x) {
    DefaultAssay(x)  <- "RNA"
    x$barcode  <- colnames(x)
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 800)
})

honar2_stacas <- Run.STACAS(merged.list, dims = 1:12)
honar2_stacas <- RunUMAP(honar2_stacas, dims = 1:12) 

In [ ]:
DimPlot(honar2_stacas)

In [ ]:
DimPlot(honar2_stacas, group.by = "Cluster_Annotation_Merged")

In [ ]:
DimPlot(honar2_stacas, group.by = "Cluster_Annotation_All")

In [ ]:
FeaturePlot(honar2_stacas, features = "FOXP3", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CD226", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CCR7", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "TNFRSF9", min.cutoff = 0)
FeaturePlot(honar2_stacas, features = "CCR10", min.cutoff = 0)

In [ ]:
FeaturePlot(honar2_stacas, features = "MKI67", min.cutoff = 0)

In [ ]:
saveRDS(honar2_stacas, "../data/published_data/Honardoost_2024/honar2_stacas.rds")